## 1. Packages & Initializations

In [1]:
from flask import Flask, request, jsonify, render_template, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import check_password_hash, generate_password_hash
from abc import ABC, abstractmethod
from statsmodels.tsa.ar_model import AutoReg
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from itertools import product
import pickle
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta

In [2]:
app = Flask(__name__)
# Configure the SQLAlchemy part of the app instance
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///aidea.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Create the SQLAlchemy db instance
db = SQLAlchemy(app)

## 2. Models

### 2.0 Model Class Abstraction

In [3]:
class Model(ABC):
    def __init__(self, data):
        self.data = data

    @abstractmethod
    def train(self):
        """
        Abstract method to train the model.
        """
        pass
    
    @abstractmethod
    def forecast(self, forecast_days):
        """
        Abstract method to make predictions using the trained model.
        """
        pass

### 2.1 AR Model

In [4]:
class AR_model(Model):
    def __init__(self, data):
        super().__init__(data)
        self.trained_model = None
        self.model_type = 'AR'

    def train(self):
        # Handle NaNs
        data = self.data.dropna(subset=['Close'])

        # Define parameter grid for tuning
        trends = ['n', 'c', 't', 'ct']
        min_lag = 1
        max_lag = len(data) 
        lags_range = range(min_lag, max_lag) 

        best_r2 = -float('inf') 
        best_params = None

        # Perform grid search with cross-validation on the training set
        # Choose the best params based on R2 score
        n_splits = 3
        tscv = TimeSeriesSplit(n_splits=n_splits)  # Time series cross-validation
        warnings.filterwarnings("ignore")
        for trend, lags in product(trends, lags_range):
            r2_sum = 0
            for train_index, val_index in tscv.split(data):
                train_split, val_split = data.iloc[train_index], data.iloc[val_index]
                try:
                    model = AutoReg(train_split['Close'].values, lags=lags, trend=trend).fit()
                    predictions = model.predict(start=len(train_split), end=len(train_split) + len(val_split) - 1)
                    r2 = r2_score(val_split['Close'], predictions)
                    r2_sum += r2
                except Exception as e:
                    print(f"Error for trend={trend}, lags={lags}: {e}")
                    continue
            
            # Average R2 score across folds
            avg_r2 = r2_sum / n_splits
            
            # Update best parameters if better R2 found
            if avg_r2 > best_r2:
                best_r2 = avg_r2
                best_params = (trend, lags)

        best_trend, best_lags = best_params
        
        print(f"Best R2 score: {best_r2:.4f}")
        print(f"Best parameters: trend={best_trend}, lags={best_lags}")
        
        # Fit the best model on the entire dataset 
        try:
            self.trained_model = AutoReg(data['Close'].values, lags=best_lags, trend=best_trend).fit()
        except Exception as e:
            print(f'Model training failed with the error message: {e}')
        
        return self.trained_model
    
    def forecast(self, forecast_days):
        #Forecast next forecast_period days
        start = len(self.data)
        end = start + forecast_days - 1
        forecast_prices = self.trained_model.predict(start=start, end=end)
        
        return forecast_prices


### Model Creation Method

In [5]:
def create_model(model_type, data):
    if model_type == 'AR':
        return AR_model(data)

## 3. Database for User and Model Tables

### 3.1 Users Table

In [6]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(255), nullable=False)
    account_type = db.Column(db.String(20), nullable=False)  # 'basic' or 'premium'

    def __repr__(self):
        return f'<User {self.email}>'

### 3.2 Trained Models Table

In [7]:
class TrainedModels(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    symbol = db.Column(db.String(20), nullable=False)
    model_type = db.Column(db.String(50), nullable=False)
    start_date = db.Column(db.String(50), nullable=False)
    end_date = db.Column(db.String(50), nullable=False)  # Nullable if model is ongoing
    trained_model = db.Column(db.Text)  # Serialized model data or file path

    def save_trained_model(self, trained_model):
        """
        Save the trained model to the database.
        """
        self.trained_model = pickle.dumps(trained_model)
        db.session.commit()
    
    def load_trained_model(self, model_type, start_date, end_date, symbol):
        model = TrainedModels.query.filter_by(
                    model_type=model_type,
                    start_date=start_date,
                    end_date=end_date,
                    symbol=symbol
                ).first()
        if model:
            return pickle.loads(model.trained_model)
        else:
            return None

## 4. Fetch Historical Data

In [8]:
def fetch_data(symbol, start_date, end_date):
    data = None
    try:
        # Fetch historical price data
        df = yf.download(symbol, start=start_date, end=end_date)

        # Drop 'Adj Close' column if present
        if 'Adj Close' in df.columns:
            df.drop(columns=['Adj Close'], inplace=True)

        # Store the dataframe with technical indicators
        data = df

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

    return data

## 5. PAGES

In [9]:
@app.route('/')
def index():
    return render_template('login.html')

### 5.1 Login 

In [10]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'GET':
        # Serve the signup page
        return render_template('login.html')

    if request.method == 'POST':
        data = request.json
        email = data.get('email')
        password = data.get('password')
        
        # Query the user from the database
        user = User.query.filter_by(email=email).first()

        if user and check_password_hash(user.password, password): # MAIN PAGE
            # If user exists and password is correct, redirect to the main page
            return jsonify({'message': 'Login successful!', 'redirect': url_for('main_page')}), 200
        else:
            # If credentials do not match, return an error
            return jsonify({'message': 'Invalid credentials'}), 401


### 5.2 Signup

In [11]:
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'GET':
        # Serve the signup page
        return render_template('signup.html')
    
    if request.method == 'POST':
        data = request.json
        if not data:
            return jsonify({'message': 'No data provided'}), 400
        
        email = data.get('email')
        password = data.get('password')
        password_re = data.get('password_re')
        
        # Check if all fields are present
        if not email or not password or not password_re:
            return jsonify({'message': 'All fields are required.'}), 400

        # Check if email already exists
        existing_user = User.query.filter_by(email=email).first()
        if existing_user:
            return jsonify({'message': 'User already exists.'}), 400

        # Check if passwords match
        if password != password_re:
            return jsonify({'message': 'Passwords do not match.'}), 400
        
        # Create new user
        hashed_password = generate_password_hash(password, method='pbkdf2:sha256')
        new_user = User(email=email, password=hashed_password, account_type='basic')

        try:
            db.session.add(new_user)
            db.session.commit()
            return jsonify({'redirect': url_for('login')}), 200
        except Exception as e:
            return jsonify({'message': 'Error creating user.'}), 500


### 5.3 Forgot Passward

In [12]:
@app.route('/forgot-password')
def forgot_password():
    # Render your forgot password page
    return 'Forgot password page'

### 5.4 Reset Password

In [13]:
@app.route('/reset-password')
def reset_password():
    # Render your forgot password page
    return 'Reset password page'

### 5.5 Predict

In [14]:
@app.route('/predict', methods=['POST'])
def predict():
    # Data fetching
    try:
        data = request.json
        symbol = data['symbol']
        data_length = data['data_length']
        forecast_days = data['forecast_days']
        model_type = data['model_type']
    
        #Set start and end date
        now = datetime.now()
        start_date =  (now - timedelta(days = data_length)).strftime("%Y-%m-%d")
        end_date = now.strftime("%Y-%m-%d")

        #Check if model already exists
        model = TrainedModels.load_trained_model(model_type=model_type, start_date=start_date, end_date=end_date, symbol=symbol)
        forecast_prices = None
        if model:  # if the model is already trained avoid re-training it.
            trained_model = model
            # Forecast
            forecast_prices = trained_model.forecast(forecast_days = forecast_days)
        else:
            data = fetch_data(symbol, start_date, end_date)
            model = create_model(model_type, data)
            trained_model = model.train()
            # Create a new instance of Model
            new_model = TrainedModels(symbol= symbol, model_type= model_type, start_date= start_date, end_date= end_date)
            # Save the trained model to the database
            new_model.save_trained_model(trained_model)
            # Commit the changes
            db.session.add(new_model)
            db.session.commit()
            # Forecast
            forecast_prices = trained_model.forecast(forecast_days = forecast_days)

        # Example: Data to send to Firebase
        processed_data = {
            'forecast_prices': forecast_prices,
            'processed': True
        }

        return jsonify(processed_data), 200 

    except KeyError as e:
        return jsonify({'error': f'Missing key: {e.args[0]}'}, 400)

    except Exception as e:
        return jsonify({'error': str(e)}, 500)

In [15]:
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Jul/2024 17:53:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2024 17:53:32] "GET /signup HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2024 17:53:52] "POST /signup HTTP/1.1" 400 -
127.0.0.1 - - [30/Jul/2024 17:54:10] "POST /signup HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2024 17:54:10] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2024 17:54:24] "POST /login HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/flask/app.py", line 1498, in __call__
    return self.wsgi_app(environ, start_response)
  File "/usr/local/lib/python3.8/site-packages/flask/app.py", line 1476, in wsgi_app
    response = self.handle_exception(e)
  File "/usr/local/lib/python3.8/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_excep